# Logging in Jupyter Notebooks with IPython

Logging is a happy thing. Let's learn how to do professional quality logging. How to properly do it with Jupyter Notebooks, and IPython? [The general Python logging infrastructure is described here](https://docs.python.org/3/library/logging.html), and is implemented by the `logging` module.

![](https://upload.wikimedia.org/wikipedia/commons/8/8a/H96566k.jpg "The first bug, Courtesy of the Naval Surface Warfare Center, Dahlgren, VA., 1988. - U.S. Naval Historical Center Online Library Photograph NH 96566-KN, Public Domain, https://commons.wikimedia.org/w/index.php?curid=165211")

## First, IPython magic (⊃｡•́‿•̀｡)⊃━☆ﾟ.*･｡ﾟ

IPython offers some [magic for logging](http://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-logstart). I don't yet have any idea if this is about logging the IPython environment itself, or is this also useful for logging application development.

* `%logstart`: activates logging to a file
* `%logstop`: stops the above
* `%logoff`: for temporarily suspending logging after it has been started
* `%logon`: for resuming suspended logging
* `%logstate`: queries the status

In [1]:
%logstart

Activating auto-logging. Current session state plus future input saved.
Filename       : ipython_log.py
Mode           : rotate
Output logging : False
Raw input log  : False
Timestamping   : False
State          : active


In [2]:
%logstate

Filename       : ipython_log.py
Mode           : rotate
Output logging : False
Raw input log  : False
Timestamping   : False
State          : active


In [3]:
%logstop

I'll try some new style print formatting, while i'm on it.

In [4]:
with open('ipython_log.py') as fd:
    loglines = fd.readlines()
    
print("Log contains total of {n_lines} lines, of which {n_unique} are unique".format(n_lines=len(loglines), n_unique=len(set(loglines))))
for line in set(loglines):
    print("{count:4}: {l}".format(count=loglines.count(line), l=line.strip()))

Log contains total of 5 lines, of which 5 are unique
   1: 
   1: get_ipython().run_line_magic('logstart', '')
   1: get_ipython().run_line_magic('logstop', '')
   1: # IPython log file
   1: get_ipython().run_line_magic('logstate', '')


## Normal Python logging

Anyhow, in Python the typical logging capabilities are provided by the `logging` module.

In [5]:
import logging

Now,

In [6]:
logging.info("hello")

does not *seem* to do anything, because IPython already has started logging, and called `logging.basicConfig`. Let's get a reference to it.

In [7]:
logger = logging.getLogger()
logger.__dict__ # standard Python object description

{'disabled': False,
 'filters': [],
 'handlers': [<StreamHandler stderr (NOTSET)>],
 'level': 30,
 'name': 'root',
 'parent': None,
 'propagate': True}

In [8]:
logger.handlers[0].__dict__

{'_name': None,
 'filters': [],
 'formatter': <logging.Formatter at 0x106584860>,
 'level': 0,
 'lock': <unlocked _thread.RLock object owner=0 count=0 at 0x1065711e0>,
 'stream': <ipykernel.iostream.OutStream at 0x1064358d0>}

Ok so that is configured to go to stderr stream, and Jupyter catches that. The default `root` logger defaults to level 30.

## Logging levels

From the [`logging` module documentation](https://docs.python.org/3/library/logging.html#levels) we learn that the logging levels are:

|Level    |Numeric value|
|---------|------------:|
|CRITICAL |50|
|ERROR    |40|
|WARNING  |30|
|INFO     |20|
|DEBUG    |10|
|NOTSET   |0|

## Creating a logger.

Ok, so the logger called *root* has been defined, and is accessible by getting it. I believe what I want is to create my own logger.

In [9]:
mylogger = logging.getLogger('mylogger')
mylogger.__dict__

{'disabled': False,
 'filters': [],
 'handlers': [],
 'level': 0,
 'manager': <logging.Manager at 0x1046a2128>,
 'name': 'mylogger',
 'parent': <RootLogger root (WARNING)>,
 'propagate': True}

In [10]:
mylogger.setLevel(logging.INFO)
mylogger.info('I is I')

INFO:mylogger:I is I


## Unclarity if it goes to the terminal, or the Notebook

Previously when I wrote this notebook, the following was be visible on the console from where IPython was conjured, or wherever this IPython happens to send it's standard error.

In [11]:
logger.critical('hello')

CRITICAL:root:hello


And indeed so it did!

![](a critical log message.png)

 This behaviour of going just to the terminal where Jupyter is running was reported as an issue [*Python log in the jupyter notebook appears only in the terminal, not in the output cell*](https://github.com/ipython/ipykernel/issues/111) for [ipykernel](https://github.com/ipython/ipykernel). Things have since changed (back to how they were earlier), and logs appear in the Notebook; unless one adds other places for them to go to. These are called *handlers*.

In [12]:
terminal_handler = logging.handlers.RotatingFileHandler('/dev/stdout')

terminal_logger = logging.Logger('terminal')
terminal_logger.addHandler(terminal_handler)

print(terminal_logger.__dict__)

{'filters': [], 'name': 'terminal', 'level': 0, 'parent': None, 'propagate': True, 'handlers': [<RotatingFileHandler /dev/stdout (NOTSET)>], 'disabled': False}


And then

In [13]:
terminal_logger.info("hello to terminal")

Of the things in `logging.handlers`, at least `RotatingFileHandler`, `QueueHandler` and `SyslogHandler` look obviously interesting.

## Integrating to workflows

Great, a new logger was greated and it does log into the console.

Now, how to tie it to the literate programming Notebook workflow? [Having a root logger is reported as an issue on IPython GitHub](https://github.com/ipython/ipython/issues/8282), which is weird since I think it's ok to have a logger. The logger framework is quite a construction on it's own.

One idea is to set the logging level of the default logger when doing developing work, like so:

In [14]:
def prime(number):
    """Is n prime?
    
    >>> prime(20047)
    True
    
    >>> prime(4)
    False
    """
    for factor in range(2, number):
        if (number % factor) == 0:
            return False
    return True

def primes(number, start=2):
    """A generator which searches n primes upwards of start.
    
    >>> list(primes(3))
    [2, 3, 5]
    
    >>> list(primes(5, start=5413))
    [5413, 5417, 5419, 5431, 5437]
    """
    count = 0
    candidate = start
    while count < number:
        logging.debug("Starting a costly computational operation, {} primes so far".format(count))
        if prime(candidate):
            # print(num, end=' ')
            yield candidate
            count += 1
        candidate += 1

Run the examples as unit tests

In [15]:
import doctest
doctest.testmod()

TestResults(failed=0, attempted=4)

Here I am using the root logger, though maybe I should adopt the habit of using my own loggers instead. I've defined `mylogger` above.

In [16]:
logging.getLogger().setLevel(logging.DEBUG)
[prime for prime in primes(3, start=3)]

DEBUG:root:Starting a costly computational operation, 0 primes so far
DEBUG:root:Starting a costly computational operation, 1 primes so far
DEBUG:root:Starting a costly computational operation, 1 primes so far
DEBUG:root:Starting a costly computational operation, 2 primes so far
DEBUG:root:Starting a costly computational operation, 2 primes so far


[3, 5, 7]

In [17]:
logging.getLogger().setLevel(logging.WARNING)
[print(prime) for prime in primes(10, start=2000000)]

2000003
2000029
2000039
2000081
2000083
2000093
2000107
2000113
2000143
2000147


[None, None, None, None, None, None, None, None, None, None]

![](http://www.lyonsinbeta.com/post-images/logging.jpg)

Two ideas were tried above: getting a new logger which defaults to level `logging.NOTSET` = 0 and using it for all logging purposes, or getting the previously existing root logger and adjusting it's level. Both seem to work. The former has the advantage that the logging messages bear a more specific name than "root".

What one does want to do further is adjust log handlers for creating f.ex. logfiles, and often getting the logging into Notebooks? Is this a useful workflow, for f.ex. data analysis jobs? On one hand logging messages are clutter, but on the other hand one wants to do literate programming. Hmm perhaps level `INFO` should come to the Notebook, and `DEBUG` should only go to the console in the background. That sounds like setting up a logging handler for `INFO` messages.